In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal, traceback
import numpy as np

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, debug=False):
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = requests.get(x)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                for s in ['/main/', '/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    x = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    if debug:
                        print(x)
                    x = requests.get(x)
                    if x.status_code == 200:
                        x = x.text
                        break
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    

###
### Display ###
###

_IMPORT('import pandas as pd')
_IMPORT('import cv2')
_IMPORT('from PIL import Image')
_IMPORT('import matplotlib.pyplot as plt')
_IMPORT('import plotly')
_IMPORT('import plotly.graph_objects as go')
_IMPORT('import ipywidgets as widgets')
_IMPORT('from ipywidgets import interact, interactive, fixed, interact_manual')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))


class COLORS(object):
    # BGR
    GREEN      = (0   , 255 , 0)
    RED        = (0   , 0   , 255)
    BLACK      = (0   , 0   , 0)
    YELLOW     = (0   , 255 , 255)
    WHITE      = (255 , 255 , 255)
    CYAN       = (255 , 255 , 0)
    MAGENTA    = (255 , 0   , 242)
    GOLDEN     = (32  , 218 , 165)
    LIGHT_BLUE = (255 , 9   , 2)
    PURPLE     = (128 , 0   , 128)
    CHOCOLATE  = (30  , 105 , 210)
    PINK       = (147 , 20  , 255)
    ORANGE     = (0   , 69  , 255)

In [2]:
import json, functools, datetime

class __JsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.datetime, datetime.timedelta)):
            return '{}'.format(obj)
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return json.JSONEncoder.default(self, obj)

json.dumps = functools.partial(json.dumps, cls=__JsonEncoder)

In [3]:
import skimage.filters as filters
import threading
from enum import IntEnum
_IMPORT('import gitee.com/qrsforever/nb_easy/easy_widget')

In [4]:
g_ctx = None

In [5]:
if g_ctx is None:
    ax = 10
else:
    ax = 20
    
print(ax)

10


## Camera Calibration

In [6]:
class CalibrateState(IntEnum):
    COLLECT = 1
    CALIBRATE = 2
    COMPLETED = 3
    

def start_camera_calibrate(
    ctx, w_btn_source, nrow, ncol, square_size, marker_size, aruto_dict,
    sample_size, w_det_objtype, w_cam_frame):
    
    ctx.logger(f'start: {nrow}, {ncol}, {square_size}, {marker_size}, {aruto_dict}, {sample_size}', clear=1)
    
    use_charuco = marker_size is not None
    if use_charuco: 
        if aruto_dict == '4x4':
            dict_id = cv2.aruco.DICT_4X4_1000
        elif aruto_dict == '5x5':
            dict_id = cv2.aruco.DICT_5X5_1000
        else:
            dict_id = cv2.aruco.DICT_6x6_1000
        charuco_dict = cv2.aruco.Dictionary_get(dict_id)
        charuco_board = cv2.aruco.CharucoBoard_create(nrow, ncol, square_size, marker_size, charuco_dict)
    else:
        # Object points for a chessboard
        objp = np.zeros((nrow * ncol, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nrow, 0:ncol].T.reshape(-1, 2)
        objp = objp * square_size
        
    pattern_size = (nrow, ncol)
    win_size, zero_zone = (5, 5), (-1, -1)
    flags=(cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_THIN_PRISM_MODEL + cv2.CALIB_TILTED_MODEL)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    detector_parameters = cv2.aruco.DetectorParameters_create()
    aruco_dict_6x6 = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_1000)
    
    def _video_capture():
        w_btn_source.disabled = True
        state = CalibrateState.COLLECT
        result = {}
        obj_points, img_points = [], []
        if use_charuco:
            charuco_corners, charuco_ids = [], []
        try:
            camera = cv2.VideoCapture(0)
            frame_width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_irate = int(camera.get(cv2.CAP_PROP_FPS))
            
            ctx.logger(f'frame_width:{frame_width} frame_height:{frame_height} display: {w_cam_frame.width}')

            ctx.camera = camera
            ctx.camera_is_running = ctx.camera.isOpened()
            
            camera_matrix, dist_coeffs, extrinsics_matrix = None, None, None
            
            def pixel_to_world(intrinsics_matrix, extrinsics_matrix, x, y):
                pseudo_inv_extrinsics = np.linalg.pinv(extrinsics_matrix)
                intrinsics_inv = np.linalg.inv(intrinsics_matrix)
                pixels_matrix = np.array((x, y, 1))
                ans = np.matmul(intrinsics_inv, pixels_matrix)
                ans = np.matmul(pseudo_inv_extrinsics, ans)
                ans /= ans[-1] 
                return ans
            
            frame_idx, iter_idx, display_frames_count = 0, 0, 1
            while ctx.camera_is_running:
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                display_frames = {'raw': frame_bgr.copy()}
                frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                if state == CalibrateState.COLLECT:
                    if use_charuco:
                        # corner detection
                        marker_corners, marker_Ids, rejected_points = cv2.aruco.detectMarkers(frame_gray, charuco_dict, parameters=detector_parameters)
                        if marker_corners is not None and len(marker_corners) > 0:
                            # coner refine 
                            marker_corners, marker_Ids, refusd, recoverd = cv2.aruco.refineDetectedMarkers(
                                frame_gray, charuco_board, marker_corners, marker_Ids, rejectedCorners=rejected_points) 
                            # corner interpolation (get charuco corners and ids from detected aruco markers)
                            retval, c_corners, c_Ids = cv2.aruco.interpolateCornersCharuco(
                                marker_corners, marker_Ids, frame_gray, charuco_board)
                            if retval > 20:
                                # Find better
                                better_charucoCorners = cv2.cornerSubPix(frame_gray, c_corners, win_size, zero_zone, criteria)                               
                                # Draw and Display markers and corners
                                frame_copy = frame_bgr.copy()
                                display_frames['det_markers'] = cv2.aruco.drawDetectedMarkers(frame_copy, marker_corners, marker_Ids)
                                frame_copy = frame_bgr.copy()
                                display_frames['det_charuco'] = cv2.aruco.drawDetectedCornersCharuco(frame_copy, better_charucoCorners, c_Ids)

                                if frame_idx % frame_irate == 0:
                                    # objp = charuco_board.chessboardCorners[charuco_Ids.flatten()]
                                    charuco_corners.append(better_charucoCorners)
                                    charuco_ids.append(c_Ids)
                                    iter_idx += 1
                                    if iter_idx == sample_size:
                                        state = CalibrateState.CALIBRATE
                    else:
                        found, corners = cv2.findChessboardCorners(frame_gray, pattern_size, flags=flags)

                        if found:
                            # Find better sub pix position for the corners in the roof corners neighbourhood
                            better_chess_corners = cv2.cornerSubPix(frame_gray, corners, win_size, zero_zone, criteria)
                            # Draw and display the corners
                            frame_copy = frame_bgr.copy()
                            frame_bgr = cv2.drawChessboardCorners(frame_copy, pattern_size, better_chess_corners, found)
                            display_frames['det_chessboard'] = frame_copy

                            if frame_idx % frame_irate == 0:
                                obj_points.append(objp)
                                img_points.append(better_chess_corners)
                                
                                iter_idx += 1
                                if iter_idx == sample_size:
                                    state = CalibrateState.CALIBRATE
                    cv2.putText(display_frames['raw'], f'Count: {iter_idx}', (70, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.GOLDEN, 1)
                    
                elif state == CalibrateState.CALIBRATE:
                    if use_charuco:
                        retval, camera_matrix, dist_coeffs, rvecs, tvecs, \
                        std_intrinsics, std_extrinsics, errors = cv2.aruco.calibrateCameraCharucoExtended(
                            charuco_corners, charuco_ids, charuco_board, frame_bgr.shape[:2], cameraMatrix=None, distCoeffs=None, flags=flags)
                        total_error = 0.0
                        for k, (c_corners, c_ids, rvec, tvec) in enumerate(zip(charuco_corners, charuco_ids, rvecs, tvecs)):
                            obj_points, img_points = cv2.aruco.getBoardObjectAndImagePoints(charuco_board, c_corners, c_ids)
                            prj_points, _ = cv2.projectPoints(obj_points, rvec, tvec, camera_matrix, dist_coeffs)
                            error = cv2.norm(img_points, prj_points, cv2.NORM_L2) / len(obj_points)
                            total_error += error
                            print(f'{k} error: {errors[k]} vs {round(error, 6)} vs {round(error**2, 6)}')
                        print(f'mean total error: {total_error / len(charuco_corners)}')
                        
                        rvec, tvec = rvecs[-1], tvecs[-1]
                    else: 
                        retval, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, frame_gray.shape[::-1], None, None)
                        obj_points, img_points = obj_points[-1], img_points[-1]

                        # Calculating extrinsics by last imgage (rotation vector)
                        retval, rvec, tvec = cv2.solvePnP(obj_points, img_points, camera_matrix, dist_coeffs)                    
                        # rvec, tvec = np.mean(np.array(rvecs), axis=0), np.mean(np.array(tvecs), axis=0)

                    # transform rotation vector to ratation matrix
                    rotation_matrix, _ = cv2.Rodrigues(rvec)
                    extrinsics_matrix = np.concatenate([rotation_matrix, tvec], 1) 

                    result['camera_matrix'] = camera_matrix
                    result['distortion_coefficients'] = dist_coeffs
                    result['extrinsics_matrix'] = extrinsics_matrix

                    print(json.dumps(result, indent=4))
                    state = CalibrateState.COMPLETED
                else: 
                    frame_copy = frame_bgr.copy()
                    display_frames['frame_undist'] = cv2.undistort(frame_bgr, camera_matrix, dist_coeffs)

                    rect = cv2.minAreaRect(img_points.reshape(-1, 2))
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    frame_copy = frame_bgr.copy()
                    display_frames['img_points_box'] = cv2.drawContours(frame_copy, [box], 0, COLORS.ORANGE, 2)


                    if w_det_objtype.value == 'ArUco_6x6': # Detect 6x6 ArUco Marker
                        marker_corners, marker_ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict_6x6, parameters=detector_parameters)
                        if np.all(marker_ids is not None):
                            frame_copy = frame_bgr.copy()
                            for (marker_corner, marker_id) in zip(marker_corners, marker_ids):
                                marker_length = 0.01 * (marker_id[0] - 60)
                                rvec, tvec, marker_points = cv2.aruco.estimatePoseSingleMarkers(marker_corner, marker_length, camera_matrix, dist_coeffs)
                                (rvec - tvec).any()
                                cv2.drawFrameAxes(frame_copy, camera_matrix, dist_coeffs, rvec, tvec, 0.1)  # Draw Axis

                                # calculate the height/width (2D to 3D)
                                (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]

                                p1 = pixel_to_world(camera_matrix, extrinsics_matrix, tl[0], tl[1])
                                p2 = pixel_to_world(camera_matrix, extrinsics_matrix, tr[0], tr[1])
                                p3 = pixel_to_world(camera_matrix, extrinsics_matrix, bl[0], bl[1])
                                p4 = pixel_to_world(camera_matrix, extrinsics_matrix, br[0], br[1])
                                W = p1 - p2
                                H = p3 - p4
                                width = np.sqrt(W[0]**2 + W[1]**2)
                                height = np.sqrt(H[0]**2 + H[1]**2)

                                cv2.putText(frame_copy,
                                            'size {} x {}'.format(round(width, 3), round(height, 3)),
                                            (tl[0], tl[1] - 15),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                            cv2.aruco.drawDetectedMarkers(frame_copy, marker_corners, marker_ids, borderColor=COLORS.CHOCOLATE)
                            display_frames['det_markers'] = frame_copy
                
                # display image
                C = len(display_frames)
                if C > 1:
                    show_ncol = 2
                    for i in range(C % show_ncol):
                        display_frames[f'placehold-{i}'] = 255 * np.ones_like(frame_bgr)
                    show_nrow = len(display_frames) // show_ncol
                    row_images = []
                    col_images = []
                    for key, img in display_frames.items():
                        cv2.putText(img, key, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS.ORANGE, 1)
                        col_images.append(img)
                        if len(col_images) == show_ncol:
                            row_images.append(np.hstack(col_images))
                            col_images = []
                    else:
                        display_image = np.vstack(row_images)
                        if C != display_frames_count:
                            w_cam_frame.layout.width = f'{int(w_cam_frame.width) * show_ncol}px'
                            w_cam_frame.layout.height = f'{int(w_cam_frame.height) * show_nrow}px'
                else:
                    display_image = display_frames.popitem()[1]
                    if C != display_frames_count:
                        w_cam_frame.layout.width = f'{w_cam_frame.width}px'
                        w_cam_frame.layout.height = f'{w_cam_frame.height}px'
                display_frames_count = C
                w_cam_frame.value = io.BytesIO(cv2.imencode('.png', display_image)[1]).getvalue()
                frame_idx += 1
                
        except Exception as err:
            ctx.logger(f'{err}: {traceback.format_exc()}')
        finally:
            w_btn_source.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()
    
    
def stop_camera_calibrate(ctx, w_btn_source):
    ctx.logger('stop_camera_calibrate()')
    if hasattr(ctx, 'camera') and ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

In [7]:
# _IMPORT('./easy_widget.py')

def nbeasy_chess_choice_objs(nrow, ncol, square_size, marker_size=-1, arUto_dict='4x4'):
    ro = True
    width = 333
    objs = [
        {
            'type': 'H',
            'objs': [
                nbeasy_widget_int('cfg.chessb_rows', 'Chess Borad Rows', nrow, width=width, readonly=ro),
                nbeasy_widget_int('cfg.chessb_cols', 'Chess Borad Cols', ncol, width=width, readonly=ro),
                nbeasy_widget_float('cfg.square_size', 'Square Size CM', square_size, width=width, readonly=ro)        
            ]
        }
    ]
    if marker_size > 0:
        enums = ['4x4', '5x5']
        objs.append({
            'type': 'H',
            'objs': [
                nbeasy_widget_float('cfg.marker_size', 'Marker Size CM', marker_size, width=width, readonly=ro),
                nbeasy_widget_stringenum(
                    'cfg.aruto_dict', 'ArUto Dict', default=enums.index(arUto_dict),
                    enums=enums,
                    width=width
                )
            ]
        })
        objs = [{'type': 'V', 'objs': objs}]
    return {
        'type': 'H',
        'objs': objs
    }
    
schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'V',
            'name': 'Camera Calibration',
            'objs': [
                nbeasy_widget_stringenumtrigger(
                    '_cfg.chess_choice', 'Chess Choise', default=5,
                    enums = ['A4-30mm-8x6', 'A4-30mm-6x8', 'A3-25mm-15x10', 'A3-25mm-10x15', 'A3-20mm-16mm-13x19-4x4', 'A4-25mm-19mm-8x11-5x5'],
                    triggers = [
                        nbeasy_chess_choice_objs(8, 6, 3.0),
                        nbeasy_chess_choice_objs(6, 8, 3.0),
                        nbeasy_chess_choice_objs(15, 10, 2.5),
                        nbeasy_chess_choice_objs(10, 15, 2.5),
                        nbeasy_chess_choice_objs(13, 19, 2.0, 1.6, '4x4'),
                        nbeasy_chess_choice_objs(8, 11, 2.5, 1.9, '5x5')
                    ],
                    width=333),
                nbeasy_widget_int('cfg.sample_size', 'Sample Size', 8, width=333)
            ]
        },
        {
            'type': 'V',
            'name': 'Camera',
            'objs': [
                {
                    'type': 'H',
                    'objs': [
                        nbeasy_widget_button('cfg.__btn_start_camera', 'Start', width=200, style='success'),
                        nbeasy_widget_button('cfg.__btn_stop_camera', 'Stop', width=200, style='success'),
                        nbeasy_widget_stringenum(
                            'cfg.det_object_type', 'Detect Object', default=0,
                            enums=['ArUco_6x6'],
                            width=333)
                    ],
                    'justify_content': 'center'
                },
                nbeasy_widget_image('__cfg.camera_frame', 'Frame', '', width=640, height=480)
            ],
            'align_items': 'center'
        },
    ],
    'evts': [
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': start_camera_calibrate,
                    'params': {
                        'sources': ['cfg.__btn_start_camera'],
                        'targets': [
                            'cfg.chessb_rows:value',
                            'cfg.chessb_cols:value',
                            'cfg.square_size:value',
                            'cfg.marker_size:value',
                            'cfg.aruto_dict:value',
                            'cfg.sample_size:value',
                            'cfg.det_object_type',
                            '__cfg.camera_frame'
                        ]
                    }
                },
                {
                    'handler': stop_camera_calibrate,
                    'params': {
                        'sources': ['cfg.__btn_stop_camera'],
                        'targets': [
                        ]
                    }
                }
            ]
        }
    ]
}

In [8]:
if g_ctx:
    if hasattr(g_ctx, 'camera') and g_ctx.camera:
        g_ctx.camera_is_running = False
        time.sleep(1)
        if g_ctx.camera:
            g_ctx.camera.release()
g_ctx = nbeasy_schema_parse(schema, debug=True)

Box(children=(Box(children=(VBox(children=(HTML(value="<b><font color='black'>Camera Calibration :</b>"), VBox…

In [9]:
raise

RuntimeError: No active exception to reraise

## UnD-1

In [ ]:
def nbon_start_camera(ctx, sbtn,
                      w_bgsub_method, w_bgsub_history, w_det_shadow, 
                      w_rmn_shadow_threshold, w_rmn_morpho_kernel, rmn_erode_iter, rmn_dilate_iter,
                      w_cnt_area_threshold,
                      w_cam_frame):
    method = str.lower(w_bgsub_method.value)
    history_length = w_bgsub_history.value
    detect_shadows = w_det_shadow.value
    
    shadow_thresh = w_rmn_shadow_threshold.value
    kernel = np.ones((w_rmn_morpho_kernel.value, w_rmn_morpho_kernel.value), np.uint8)
    
    erode_iter = rmn_erode_iter.value
    dilate_iter = rmn_dilate_iter.value
    
    area_threshold = w_cnt_area_threshold.value
    
    ctx.logger(f'nbon_start_camera({method}, {history_length}, {detect_shadows}, {shadow_thresh}, {w_rmn_morpho_kernel.value})')
    
    import threading
    
    def _video_capture():
        sbtn.disabled = True
        parameters = cv2.aruco.DetectorParameters_create()
        aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)
        pixel_cm_ratio_1, pixel_cm_ratio_2 = -1, -1
        marker_corner_1, marker_corner_2 = None, None
        try:
            bg_object = None
            if method == 'knn':
                bg_object = cv2.createBackgroundSubtractorKNN(history=history_length, detectShadows=detect_shadows)
            elif method == 'mog2':
                bg_object = cv2.createBackgroundSubtractorMOG2(history=history_length, detectShadows=detect_shadows)

            camera = cv2.VideoCapture(0)
            width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))

            ctx.camera = camera
            ctx.camera_is_running = ctx.camera.isOpened()
            while ctx.camera_is_running:
                ret, frame_bgr = camera.read()
                if not ret:
                    break
                    
                # Detect Aruco markers 5cm x 5cm
                corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
                if corners is None or len(corners) == 0: 
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                    
                frame_copy = frame_bgr.copy()
                    
                for (marker_corner, marker_ID) in zip(corners, ids):
                    (tl, tr, br, bl) = [(int(x[0]), int(x[1])) for x in marker_corner.reshape((4, 2))]
                    rect = cv2.minAreaRect(marker_corner[0])
                    if marker_ID == 1:
                        pixel_cm_ratio_1 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target left
                        marker_corner_1 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:1 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_1[0][0], marker_corner_1[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                    elif marker_ID == 2:
                        pixel_cm_ratio_2 = cv2.arcLength(marker_corner, True) / (4 * 5) # place it on the target right
                        marker_corner_2 = (tl, tr, br, bl)
                        cv2.putText(frame_copy,
                                    'id:2 {}x{}'.format(round(rect[1][0], 2), round(rect[1][1], 2)),
                                    (marker_corner_2[0][0], marker_corner_2[0][1] - 15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                        
                if marker_corner_1 is None and marker_corner_2 is None:
                    w_cam_frame.value = io.BytesIO(cv2.imencode('.png', frame_bgr)[1]).getvalue()
                    continue
                                                             
                cv2.aruco.drawDetectedMarkers(frame_copy, corners, ids, borderColor=(255, 0, 0))

                if bg_object:
                    # Apply the background object on the frame to get the segmented mask.     
                    fgmask = bg_object.apply(frame_bgr)

                    # Perform thresholding to get rid of the shadows.
                    _, fgmask = cv2.threshold(fgmask, shadow_thresh, 255, cv2.THRESH_BINARY)

                    # Apply some morphological operations to make sure you have a good mask
                    fgmask = cv2.erode(fgmask, kernel, iterations=erode_iter)
                    fgmask = cv2.dilate(fgmask, kernel, iterations=dilate_iter)

                    # Get foreground object
                    foreground = cv2.bitwise_and(frame_bgr, frame_bgr, mask=fgmask)
                    
                    frame_mid = foreground
                    frame_dst = fgmask
                else:
                    # Grayscale & Guassian blur
                    frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY) 
                    frame_blur = cv2.GaussianBlur(frame_gray, (3, 3), 0)

                    # # Divide gray by morphology image
                    # frame_div = cv2.divide(frame_gray, frame_blur, scale=255)

                    # Sharpen using unsharp masking
                    # frame_sharp = filters.unsharp_mask(frame_div, radius=1.5, amount=1.5, channel_axis=None, preserve_range=False)
                    # frame_sharp = (255 * frame_sharp).clip(0, 255).astype(np.uint8)

                    # Otsu Filter
                    # _, frame_otsu = cv2.threshold(frame_blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    _, frame_otsu = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_OTSU)

                    # Detect edge by candy
                    # frame_canny = cv2.Canny(frame_blur, 200, 250)
                    
                    frame_mid = cv2.cvtColor(frame_otsu, cv2.COLOR_GRAY2BGR)
                    frame_dst = frame_otsu

                contours, _ = cv2.findContours(frame_dst, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                for cnt in contours:
                    if cv2.contourArea(cnt) > area_threshold:
                        x1, y1, width, height = cv2.boundingRect(cnt)
                        x2, y2 = x1 + width, y1 + height
                        
                        cv2.rectangle(frame_copy, (x1, y1), (x2, y2),(0, 0, 255), 2)
                        cv2.putText(frame_copy,
                                    '{}x{}'.format(round(width, 2), round(height, 2)),
                                    (x2, y2),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                        
                        if marker_corner_1:
                            if x1 < marker_corner_1[1][0] or x1 < marker_corner_1[2][0]:
                                continue
                        if marker_corner_2:
                            if x2 > marker_corner_2[0][0] or x2 > marker_corner_2[3][0]:
                                continue
                        
                        rect = cv2.minAreaRect(cnt)
                        (cx, cy), (cw, ch), angle = rect
                        
                        if marker_corner_1 and marker_corner_2:
                            ow = round(0.5 * cw / pixel_cm_ratio_1 + 0.5 * cw / pixel_cm_ratio_2, 2)
                            oh = round(0.5 * ch / pixel_cm_ratio_1 + 0.5 * ch / pixel_cm_ratio_2, 2)
                        elif marker_corner_1:
                            ow = round(cw / pixel_cm_ratio_1, 2)
                            oh = round(ch / pixel_cm_ratio_1, 2)
                        else:
                            ow = round(cw / pixel_cm_ratio_2, 2)
                            oh = round(ch / pixel_cm_ratio_2, 2)
                        
                        cv2.circle(frame_copy, (int(cx), int(cy)), 10, (255, 0, 0), -1)
                        cv2.polylines(frame_copy, [np.int0(cv2.boxPoints(rect))], True, (255, 0, 0), 2)
                        cv2.putText(frame_copy, "{}x{}".format(ow, oh), (int(cx - 100), int(cy - 20)), cv2.FONT_HERSHEY_PLAIN, 0.5, (100, 200, 0), 1)

                stacked = np.hstack((frame_bgr, frame_mid, frame_copy))
                w_cam_frame.value = io.BytesIO(cv2.imencode('.png', stacked)[1]).getvalue()
                
        except Exception as err:
            ctx.logger(f'{err}')
        finally:
            sbtn.disabled = False
            ctx.camera.release()
            ctx.camera_is_running = False
            ctx.camera = None
        
    
    camera_thread = threading.Thread(target=_video_capture, name='camera')
    camera_thread.daemon = True
    camera_thread.start()

    
def nbon_stop_camera(ctx, sbtn):
    ctx.logger('nbon_stop_camera()')
    if ctx.camera:
        ctx.camera_is_running = False
        for i in range(3):
            if ctx.camera is None:
                break
            time.sleep(1)
        if ctx.camera:
            ctx.camera.release()

In [ ]:
schema = {
    'type': 'page',
    'objs': [
        {
            'type': 'tab',
            'objs': [
                {
                    'name': 'Configuration',
                    'objs': [
                        {
                            'type': 'H',
                            'name': 'Background Subtraction',
                            'objs': [
                                nbeasy_widget_stringenum('cfg.bgsub_method', 'BG Sub Method', 1, enums=['None', 'MOG2', 'KNN'], width=300),
                                nbeasy_widget_int('cfg.bgsub_history', 'History Length', '300', min_=100, max_=600),
                                nbeasy_widget_bool('cfg.bgsub_det_shadow', 'Detect Shadows', True)
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Remove Noise',
                            'objs': [
                                nbeasy_widget_int('cfg.rmn_shadow_threshold', 'Shadow Threshold', 250, min_=1, max_=255),
                                nbeasy_widget_stringenum('cfg.rmn_morpho_kernel', 'Morpho Kernel', 0, enums=[('3', 3), ('5', 5), ('7', 7), ('9', 9)], width=300),
                                nbeasy_widget_int('cfg.rmn_erode_iter', 'Erode Iters', 1),
                                nbeasy_widget_int('cfg.rmn_dilate_iter', 'Dilate Iters', 1),
                            ]
                        },
                        {
                            'type': 'H',
                            'name': 'Find Contours',
                            'objs': [
                                nbeasy_widget_float('cfg.cnt_area_threshold', 'Area Threshold', 2000),
                            ]
                        },
                        {
                            'type': 'V',
                            'name': 'Camera',
                            'objs': [
                                {
                                    'type': 'H',
                                    'objs': [
                                        nbeasy_widget_button('cfg.btn_start_camera', 'Start', width=200, style='success'),
                                        nbeasy_widget_button('cfg.btn_stop_camera', 'Stop', width=200, style='success')
                                    ],
                                    'justify_content': 'center'
                                },
                                nbeasy_widget_image('__cfg.camera_frame', 'Frame', '', height=480)
                            ],
                            'align_items': 'center'
                        }
                    ]
                }
            ]
        },
        {
            'type': 'onclick',
            'objs': [
                {
                    'handler': nbon_start_camera,
                    'params': {
                        'sources': ['cfg.btn_start_camera'],
                        'targets': [
                            'cfg.bgsub_method',
                            'cfg.bgsub_history',
                            'cfg.bgsub_det_shadow',
                            'cfg.rmn_shadow_threshold',
                            'cfg.rmn_morpho_kernel',
                            'cfg.rmn_erode_iter',
                            'cfg.rmn_dilate_iter',
                            'cfg.cnt_area_threshold',
                            '__cfg.camera_frame'
                        ]
                    }
                },
                {
                    'handler': nbon_stop_camera,
                    'params': {
                        'sources': ['cfg.btn_stop_camera'],
                        'targets': [
                        ]
                    }
                }
            ]
        }
    ]
}

if G:
    if hasattr(easy, 'camera') and easy.camera:
        easy.camera_is_running = False
        time.sleep(1)
        if easy.camera:
            easy.camera.release()
easy = nbeasy_schema_parse(schema, debug=True)
G = easy

In [ ]:
raise

## UnDef-2

In [ ]:
from IPython.display import display, Image

In [ ]:
parameters = cv2.aruco.DetectorParameters_create()
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)

In [ ]:
display_handle = display(None, display_id=True)

try:
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(width, height)
    ret, prev_frame= cap.read()
    while True:
        _, frame_bgr = cap.read()
        
        # frame_diff = cv2.absdiff(frame_bgr, prev_frame)
        # prev_frame = frame_bgr.copy()
        # gray = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        # blur = cv2.GaussianBlur(gray, (5, 5), 0)   
        # thresh = cv2.threshold(blur, 10, 255, cv2.THRESH_BINARY)[1]
        # dilate = cv2.dilate(thresh, None, iterations=5)
        # contours = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

        # cv2.drawContours(frame_bgr, contours, -1, (0, 0, 0), 3)

        motion = 0
        # for cnt in contours:
        #     mask = np.zeros([height, width], dtype=np.uint8)
        #     area = cv2.contourArea(cnt)
        #     if area > 10000:
        #         motion += 1
        #         # rect = cv2.minAreaRect(cnt)
        #         # (x, y), (w, h), angle = rect
        #         # object_width = w / pixel_cm_ratio
        #         # object_height = h / pixel_cm_ratio
        #         x, y, w, h = cv2.boundingRect(cnt)
        #         x1 = x if x < 5 else x - 5
        #         y1 = y if y < 5 else y - 5
        #         x2 = x + w if (x + w + 5) > width else x + w + 5
        #         y2 = y + h if (y + h + 5) > height else y + h + 5
        #         # cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (255, 255, 255), -1)
        #         mask[y1:y2, x1:x2] = 255
        #         frame_mask = cv2.add(frame_bgr, np.zeros_like(frame_bgr, dtype=np.uint8), mask=mask)
                  
        if motion > -1:
            corners, ids, _ = cv2.aruco.detectMarkers(frame_bgr, aruco_dict, parameters=parameters)
            # cv2.aruco.drawDetectedMarkers(frame_bgr, corners, ids)
            if corners and len(corners) > 0: 
                aruco_perimeter = cv2.arcLength(corners[0], True)
                pixel_cm_ratio = aruco_perimeter / 20
                frame_gray = cv2.cvtColor(frame_bgr.copy(), cv2.COLOR_BGR2GRAY)
                img_blur = cv2.GaussianBlur(frame_gray, (5, 5), 0)
                img_edged = cv2.Canny(img_blur, 50, 100)
                img_edged = cv2.dilate(img_edged, None, iterations=1)
                img_edged = cv2.erode(img_edged, None, iterations=1)
                # frame_thresh = cv2.adaptiveThreshold(frame_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 19, 5)
                contours = cv2.findContours(img_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    # if area > 5000 and area < 153600:
                    if area > 1000 and area < 20000:
                        # cv2.drawContours(frame_bgr, [cnt], 0, color=(0, 0, 0), thickness=4)
                        rect = cv2.minAreaRect(cnt)
                        (x, y), (w, h), angle = rect
                        object_width = w / pixel_cm_ratio
                        object_height = h / pixel_cm_ratio
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
 
                        cv2.circle(frame_bgr, (int(x), int(y)), 10, (0, 0, 255), -1)
                        cv2.polylines(frame_bgr, [box], True, (100, 10, 0), 2)
                        # cv2.putText(frame_bgr, "{} cm".format(round(object_width, 2)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
                        cv2.putText(frame_bgr, "{} cm".format(round(object_height, 2)), (int(x - 50), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)
        
        _, frame = cv2.imencode('.jpeg', frame_bgr)
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    cap.release()
    display_handle.update(None)

## References

- [旋转矩阵](https://slash-honeydew-c53.notion.site/a88e94293aeb4b54a729ceeb2f40a353)

1. Part 1: Image formation and pinhole model of the camera - https://towardsdatascience.com/image-formation-and-pinhole-model-of-the-camera-53872ee4ee92
2. Part 2: Camera Extrinsic Matrix in Python - https://towardsdatascience.com/camera-extrinsic-matrix-with-example-in-python-cfe80acab8dd
3. Part 3: Camera Intrinsic Matrix in Python - https://towardsdatascience.com/camera-intrinsic-matrix-with-example-in-python-d79bf2478c12
4. Part 4: Find the Minimum Stretching Direction of Positive Definite Matrices - https://towardsdatascience.com/find-the-minimum-stretching-direction-of-positive-definite-matrices-79c2a3b397fc
5. Part 5: Camera Calibration in Python - https://towardsdatascience.com/camera-calibration-with-example-in-python-5147e945cdeb
You can also find all the code in the GitHub repository - https://github.com/wingedrasengan927/Image-formation-and-camera-calibration


https://programtalk.com/vs4/python/OteRobotics/realant/pose_estimation.py/